<a href="https://colab.research.google.com/github/v1umahmo/HonoursProject/blob/main/HonoursCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Necessary Libraries

In [1]:
#For data processing
import pandas as pd
#For linear algebra
import numpy as np
#Regular expression library
import re
import nltk
import seaborn as sns
import warnings
import matplotlib.pyplot as plt


### Import the Training and Testing Data

In [22]:
train = pd.read_csv("/content/train.csv") # Import training data from file path
test = pd.read_csv("/content/test.csv") # Import test data from file path

### Look At The Training Data

In [23]:
train.info() # Print training data info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [24]:
train.head(10) # Print the first ten tweets from the training data

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


In [25]:
train.tail(10) # Print the last ten tweets from the training data

,id,label,tweet
31952,31953,0,@user you went too far with @user
31953,31954,0,good morning #instagram #shower #water #berlin...
31954,31955,0,#holiday bull up: you will dominate your bul...
31955,31956,0,less than 2 weeks ð ðð¼ð¹ððµ @us...
31956,31957,0,off fishing tomorrow @user carnt wait first ti...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


### Clean The Training Data

In [26]:
# For loop to go through each tweet for the data cleaning
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, ' ', input_txt)
  return input_txt

In [27]:
# Get Rid of the user handles
train['cleaned_tweets'] = np.vectorize(remove_pattern)(train['tweet'], "@[\w]*")

In [28]:
train.head(10) # Print the first ten tweets from the training data with their twitter handles removed

,id,label,tweet,cleaned_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ¦
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here ! i'm it's so #gr8 !


In [29]:
# Make tweets lower case
train['cleaned_tweets'] = train['cleaned_tweets'].apply(lambda x: x.lower())

In [30]:
train.head(10) # Print the first ten tweets from the training data with the tweets lower case

,id,label,tweet,cleaned_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ¦
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here ! i'm it's so #gr8 !


In [31]:
# Remove hashtags and special characters
train['cleaned_tweets'] = train['cleaned_tweets'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))

In [32]:
train.head(10)

,id,label,tweet,cleaned_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,2 2 huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ð ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here i m it s so gr8


In [33]:
# Remove numbers etc.
train['cleaned_tweets'] = train['cleaned_tweets'].apply(lambda x: re.sub(r'[^a-zA-z]',' ',x))

In [34]:
train.head(10)

,id,label,tweet,cleaned_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here i m it s so gr


In [42]:
# Import libraries for tokenization and stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
# Create tokenization for the cleaned tweets
train['tokenized_tweets'] = train['cleaned_tweets'].apply(lambda x: word_tokenize(x))

In [44]:
train.head(10)

,id,label,tweet,cleaned_tweets,tokenized_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...,"[when, a, father, is, dysfunctional, and, is, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause ...,"[thanks, for, lyft, credit, i, can, t, use, ca..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...,"[model, i, love, u, take, with, u, all, the, t..."
4,5,0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]"
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...,"[huge, fan, fare, and, big, talking, before, t..."
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny,"[camping, tomorrow, danny]"
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...,"[the, next, school, year, is, the, year, for, ..."
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...,"[we, won, love, the, land, allin, cavs, champi..."
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here i m it s so gr,"[welcome, here, i, m, it, s, so, gr]"


In [40]:
# Use this to remove words such as if, but, can etc. (Words that have no value)
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words

LookupError: ignored

In [ ]:
# Apply stopwords to the tokenized tweets
train['tokenized_tweets_filtered'] = train['tokenized_tweets'].apply(lambda x: [word for word in x if not word in stop_words])

In [ ]:
train.head(10)

In [ ]:
#Import library for stemming
from nltk.stem import PorterStemmer
stemming = PorterStemmer()

In [ ]:
#Stem the tweets and create another colim for tweets that are stemmed
train['stemmed_tweets'] = train['tokenized_tweets_filtered'].apply(lambda x: ' '.join([stemming.step(i) for i in x]))

In [ ]:
train.head(10)

In [ ]:
#Import library for lemmatizing
from nltk.stem.wordnet import WordnetLemmatizer
lemmatizing = WordnetLemmatizer()

In [ ]:
# Lemmatize the tweets and create another column for tweets that are lemmatized
train['lemmatized_tweets'] = train['tokenized_tweets_filtered'].apply(lambda x: ' ' .join([lemmatizing.lemmatize(i) for i in x]))

In [ ]:
train.head(10)

### Apply Feature Extraction

In [ ]:
# Import library
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tf_idf 

In [ ]:
traintfidf_stem = tf_idf.fit_transform(train['tweet_stemmed'])
traintfidf_stem.toarray()

In [ ]:
testtfidf_lemm = tf_idf.fit_transform(train['tweet_lemmatized'])
testtfidf_lemm.toarray()

### Clean The Test Data

In [ ]:
test.info()

In [ ]:
test.head(10)

In [ ]:
test.tail(10)

In [ ]:
# Get Rid of the user handles
test['cleaned_tweets'] = np.vectorise(remove_pattern)(test['tweet'], "@[\w]*")

In [ ]:
test.head(10)

In [ ]:
# Make tweets lower case
test['cleaned_tweets'] = test['cleaned_tweets'].apply(lambda x: x.lower())

In [ ]:
test.head(10)

In [ ]:
test['cleaned_tweets'] = test['cleaned_tweets'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))

In [ ]:
test.head(10)

In [ ]:
test['cleaned_tweets'] = test['cleaned_tweets'].apply(lambda x: re.sub(r'[^a-zA-z]',' ',x))

In [ ]:
test.head(10)

In [ ]:
# Import libraries for tokenization and stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Create tokenization for the cleaned tweets
test['tokenized_tweets'] = test['cleaned_tweets'].apply (lambda x: word_tokenize(x))

In [ ]:
test.head(10)

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words

In [ ]:
test['tokenized_tweets_filtered'] = test['tokenized_tweets'].apply(lambda x: [word for word in x if not word in stop_words])

In [ ]:
test.head(10)

In [ ]:
#Import library for stemming
from nltk.stem import PorterStemmer
stemming = PorterStemmer()

In [ ]:
#Stem the tweets and create another colim for tweets that are stemmed
test['stemmed_tweets'] = test['tokenized_tweets_filtered'].apply(lambda x: ' '.join([stemming.step(i) for i in x]))

In [ ]:
test.head(10)

In [ ]:
#Import library for lemmatizing
from nltk.stem.wordnet import WordnetLemmatizer
lemmatizing = WordnetLemmatizer()

In [ ]:
test['lemmatized_tweets'] = test['tokenized_tweets_filtered'].apply(lambda x: ' ' .join([lemmatizing.lemmatize(i) for i in x]))

In [ ]:
test.head(10)

### Apply Feature Extraction

In [ ]:
# Import library
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tf_idf 

In [ ]:
testtfidf_stem = tf_idf.fit_transform(test['tweet_stemmed'])
testtfidf_stem.toarray()

In [ ]:
testtfidf_lemm = tf_idf.fit_transform(train['tweet_lemmatized'])
testtfidf_lemm.toarray()

### Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklear.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

### Train & Test Split

In [ ]:
x = traintf_idf
y = train['label']

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size = 0.3,random_state = 42)

### Applying SVM using Lemmatized Data

In [ ]:
svc = SVC()
svc.fit(xtrain,ytrain)
predict_svc = svc.predict(xtest)

In [ ]:
print(accuracy_score(predict_svc,ytest))
print(confusion_matrix(predict_svc,ytest))

### Applying Naïve Bayes using Lemmatized Data

In [ ]:
na_bay = GaussianNB()
na_bay.fit(xtrain.toarray(),ytrain)
predict_na_bay = na_bay.predict(xtest.toarray())

In [ ]:
print(accuracy_score(predict_na_bay,ytest))
print(confusion_matrix(predict_na_bay,ytest))

### Applying Logistic Regression using Lemmatized Data

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(xtrain,ytain)
predict_log_reg = log_reg.predict(xtest)

In [ ]:
print(accuracy_score)
print(confusion_matrix)

### Applying Machine Learning Algorithm using Stemmed Data

### Accuracy and Confusion Matrix